<h1>ELECTRE Tri analysis of a building energy retrofit</h1>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/cghiaus/ELECTRE_Tri/tree/main/HEAD?labpath=docs%2Ftutorials%2FELECTRE_Tri_tutorial.ipynb)

ELECTRE Tri-B is a multiple-criteria decision-making (MCDM) method designed to sort a set of alternatives into predefined ordered categories based on their performance across multiple criteria.

Let's consider the problem of sorting three alternatives of energy retrofit of a building.

**Given:**

- A set of criteria $\{c_{k}\}$:
    - __c1: Saving (in kWh/m²/year)__ with weight $w = 0.7$
    - __c2: Cost (in €/m²)__ with weight $w = 0.3$
___
- A set of alternatives $\{a_{i}\}$:
   - __a1: Basic renovation__
   - __a2: Moderate renovation__
   - __a3: Extensive renovation__
___
- A matrix $A = \{a_{i,k}\}$ of performances of alternatives $a_i$ for each criterion $c_k$.

| Alternative profile | c1: Saving/(kWh/m²/year)| c2: Cost/(€/m²)|
|---------------------------|------------------:|------------:|
| a1: Basic renovation      | 50.0              | 100.0       |
| a2: Moderate renovation   | 80.0              | 200.0       |
| a3: Extensive renovation  | 120.0             | 350.0       |
___

- A matrix $B = \{b_{j,k}\}$ of base profiles (or category boundaries) $b_j$ for each criterion $k$.

| Base profile | Saving/(kWh/m²/year) | Cost/(€/m²) |
|---------|---------------------:|-------------:|
| b1: bad | 50.0                 |   300.0      |
| b2: good| 100.0                |   100.0      |

These base profiles form a set of $n + 1$ ordered __categories__ $C$, where $n$ is the number of base profiles $B$ in which the alternatives are to be sorted:

- __b1>__ worse than b1 (the lower base profile) 
- __(b1, b2)__ between base profiles __b1__ and __b2__;
- ... 
- __bn<__ better than the best profile __bn__.
____

- A matrix $T$ of thresholds for:
  - indifference $q = \{q_k\}$, the largest difference in performance on a criterion that a decision-maker considers insignificant;
  - preference $p = \{p_k\}$,  the smallest difference in performance on a criterion that the decision-maker considers significant enough to strongly prefer one alternative over another;
  - veto $v = \{v_k\}$, the maximum allowable difference in performance on a criterion beyond which an alternative cannot outrank another, regardless of its performance on other criteria.

| Threshold profile | Saving/(kWh/m²/year) | Cost/(€/m²) |
|---------|---------------------:|------------:|
| q       | 5.0                  | 10.0        |
| p       | 10.0                 | 25.0        |
| v       | 20.0                 | 50.0        |
___

- A credibility threshold $\lambda$ for outranking, i.e. minimum degree of credibility index that is considered necessary to validate the statement "alternative a outranks base profile b" (value within the range [0.5, 1], typically 0.75).
____

**Do:**

Assign each alternative $a_i ∈ A$ to one of the predefined categories $C_j ∈ C$ based on its performance across all criteria $c_k$ by using optimistic and pessimistic classification:

| Categories   | Alternatives                             |
|--------------|------------------------------------------|
| __b1 ≻__     | [list of alternatives worse than b1]     |
| __(b1, b2)__ | [list of alternatives between b1 and b2] |
| __b2 ≺__     | [list of alternatives better than b2]    |

____

__Procedure:__
1. Upload your data file containing the performance of alternatives, base profiles, thresholds, and weights.
2. Read data file and solve ELECTRE Tri-B sorting problem.
3. Interpret the results.

In [1]:
"""
Append `src/` directory to `path`
"""
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(os.path.dirname(notebook_dir))

src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)

In [2]:
import pandas as pd

import electre_tri as et

# Data file

ELECTRE Tri-B data is a [.csv](https://en.m.wikipedia.org/wiki/Comma-separated_values) text file that contains, for each criterion:
- performance of alternatives **A**;
- base profiles **B**;
- indifference, preference and veto thresholds **T**;
- weights **w**. 

__Note__: For criteria to be minimized (i.e. cost), the data for performance of alternatives __A__ and the base profiles __B__ are negative numbers.

Data files can be written by using a [spreadsheet](https://en.m.wikipedia.org/wiki/Spreadsheet).

In [3]:
data_file = "../../data/bldg_retrofit_base.csv"
print("Example of data file")
pd.read_csv(data_file)

Example of data file


,type,profile,Saving/(kWh/m²/year),Cost/(€/m²)
0,A,a1: Basic renovation,50.0,-100.0
1,A,a2: Moderate renovation,80.0,-200.0
2,A,a3: Extensive renovation,120.0,-350.0
3,B,bad,50.0,-300.0
4,B,good,100.0,-100.0
5,T,q,5.0,10.0
6,T,p,10.0,25.0
7,T,v,20.0,50.0
8,w,NaN,0.7,0.3


# Problem solving

In [4]:
# Problem statement
data_file = "../../data/simple_example.csv"
credibility_threshold = 0.7

# Problem solving
A, B, T, w = et.read_electre_tri_data(data_file)
optimistic, pessimistic = et.electre_tri_b(A, B, T, w,
                                           credibility_threshold)

# Results

In [5]:
# Optimistic sorting
opti_sort = et.sort(optimistic)
print('Optimistic sorting')
opti_sort.to_frame(name="alternatives").rename_axis("categories")

Optimistic sorting


,alternatives
categories,
b1 ≻,[a1]
"(b1, b2)",[]
b2 ≺,"[a2, a3]"


In [6]:
# Pessimistic sorting
pessi_sort = et.sort(pessimistic)
print('Pessimistic sorting')
pessi_sort.to_frame(name="alternatives").rename_axis("categories")

Pessimistic sorting


,alternatives
categories,
b1 ≻,"[a1, a3]"
"(b1, b2)",[a2]
b2 ≺,[]
